In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import classification_report
pd.set_option("display.max_columns",None)
#pd.set_option("display.max_rows",None)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
bc=pd.read_csv(r"C:\Users\vaibhav\Desktop\train_s3TEQDk.csv")
bc

In [ ]:
bc.shape

In [ ]:
bc.isnull().sum()

In [ ]:
bc.Is_Lead.value_counts()

In [ ]:
bc['Credit_Product']=bc['Credit_Product'].fillna('No')

In [ ]:
bc2=bc.drop(['ID','Credit_Product','Gender'], axis=1)
bc2

In [ ]:
import sklearn
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [ ]:
bc2[bc2.select_dtypes(include=['object']).columns]=bc2[bc2.select_dtypes(include=['object']).columns].apply(le.fit_transform)

In [ ]:
bc_x_train=bc2.iloc[:,0:7]
bc_y_train=bc2.iloc[:,-1]

****Testing **** Set

In [ ]:
test=pd.read_csv(r"C:\Users\vaibhav\Desktop\test_mSzZ8RL.csv")

In [ ]:
test.head()

In [ ]:
test.shape

In [ ]:
test.info()

In [ ]:
test.isnull().sum()

In [ ]:
test['Credit_Product']=test['Credit_Product'].fillna('No')

In [ ]:
test2=test.drop(['ID','Credit_Product','Gender'], axis=1)
test2

In [ ]:
import sklearn
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [ ]:
test2[test2.select_dtypes(include=['object']).columns]=test2[test2.select_dtypes(include=['object']).columns].apply(le.fit_transform)

In [ ]:
test2_x=test2.loc[:,:]

SMOTE

In [ ]:
import math
import imblearn
from imblearn.over_sampling import SMOTE

In [ ]:
smt=SMOTE()
bc_x_train,bc_y_train=smt.fit_resample(bc_x_train,bc_y_train)

In [ ]:
bc_y_train.value_counts()

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
logmodel=LogisticRegression()
logmodel.fit(bc_x_train,bc_y_train)
pred_log=logmodel.predict(test2_x)

K nearest neighbours

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=30)
knn.fit(bc_x_train,bc_y_train)
pred_knn=knn.predict(test2_x)

Xg boost

In [ ]:
import xgboost
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
model = XGBClassifier()
model.fit(bc_x_train,bc_y_train)
predict_model = model.predict(test2_x)

In [ ]:
df=pd.DataFrame(predict_model)
df

In [ ]:
final_df = pd.concat([test.ID, df], axis =1)
final_df

In [ ]:
final_df.rename(columns = {0 : "Is_Lead"}, inplace = True)
final_df

In [ ]:
final_df.to_csv(r'submission.csv',index=False)

In [ ]:
smt=SMOTE()
bc_x_train,bc_y_train=smt.fit_resample(bc_x_train,bc_y_train)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
rf=RandomForestClassifier()
rfe_rfc=RFE(rf,7)
rfe_rfc.fit(bc_x_train,bc_y_train)
rfe_rfc.support_    #below isthe output 

In [ ]:
feature_importance=pd.DataFrame({'Features':list(bc_x_train),'Importance':list(rfe_rfc.support_)})
feature_importance

Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtree_fi = DecisionTreeClassifier()
dtree_fi.fit(bc_x_train,bc_y_train)
dfi = pd.DataFrame({"Feauture": bc_x_train.columns, "Importance": dtree_fi.feature_importances_})
dfi

RandomForest Classifier

In [ ]:
rfc_fi = RandomForestClassifier()
rfc_fi.fit(bc_x_train,bc_y_train)
firf = pd.DataFrame({"Feauture": bc_x_train.columns, "Importance": rfc_fi.feature_importances_})
firf

Feature Importance

In [ ]:
Final_feature_selection = pd.DataFrame({"Feauture": bc_x_train.columns, "RFE": rfe_rfc.support_,
                                  "Importance(Decision Tree)": dtree_fi.feature_importances_, "Importance (RandomForest)": rfc_fi.feature_importances_,
                                })
Final_feature_selection=Final_feature_selection.sort_values(by=['Importance(Decision Tree)'], ascending=False)
Final_feature_selection

GradientBoostingClassifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(random_state=100)
clf.fit(bc_x_train, bc_y_train)
pred_gb=clf.predict(test2_x)

In [ ]:
df=pd.DataFrame(pred_log)
df

In [ ]:
final_df = pd.concat([test.ID, df], axis =1)
final_df

In [ ]:
final_df.rename(columns = {0 : "Is_Lead"}, inplace = True)
final_df

In [ ]:
final_df.to_csv(r'submission.csv',index=False)